# Welcome to the Challenge Trading Planner
This calculator is to helo traders who engage in tiny account trading challenges.

For those who discover them, tiny account trading challenges are about practicing trading technics by using very small balances (<100 USD or equivalent) and grow them to multiple factors over the course of 2-4 weeks.

Achieving this is tied to:
- Good understanding of the charts to find good entries/exits/stop losses
- Good management of risk.

This calculator helps on the latter part.

## Projection Calculation

In [85]:
# ---- Step 1: Get user input with defaults ----
starting_capital = 10
rate = 20  # in percent
periods = 30

print(f"Starting Capital: {starting_capital}")
print(f"Period count: {periods}")
print(f"Period gain rate: {rate}")

Starting Capital: 10
Period count: 30
Period gain rate: 20


In [86]:
import pandas as pd

# ---- Step 2: Calculate compounded values ----
data = []
ultimate_gain = 0
previous_value = starting_capital
for period in range(1, periods + 1):
    value = starting_capital * ((1 + rate / 100) ** period)
    delta = value - previous_value
    previous_value = value
    ultimate_gain = value # saving to get the last value
    data.append({"Period": period, "Value": round(value, 2), "Delta": round(delta, 2)})


# ---- Step 3: Create DataFrame and print ----
df = pd.DataFrame(data)

# Show all rows
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)

# Optional: prevent column width truncation
pd.set_option('display.max_colwidth', None)

# print
df


,Period,Value,Delta
0,1,12.00,2.00
1,2,14.40,2.40
2,3,17.28,2.88
3,4,20.74,3.46
4,5,24.88,4.15
5,6,29.86,4.98
6,7,35.83,5.97
7,8,43.00,7.17
8,9,51.60,8.60
9,10,61.92,10.32


Delta here can be interpreted as a per-period goal.

In [87]:
ultimate_gain = round(ultimate_gain, 2)
ultimate_growth = round(ultimate_gain /starting_capital * 100, 2)
print(f"Ultimate Gain Target: {ultimate_gain} currency -> {ultimate_growth} %")

Ultimate Gain Target: 2373.76 currency -> 23737.6 %


## Risk parameters

In [88]:
rr = 4 ## Risk Reward
max_margin_account_rate = 10 # max % of account put into a position at any given time
max_account_rate_at_risk = 1 # max % of account losable per trade
print(f"Risk/Reward: 1:{rr}")
print(f"Max Margin per Trade: {max_margin_account_rate} % of account")
print(f"Max Losable: {max_account_rate_at_risk}% of account")
print("")



Risk/Reward: 1:4
Max Margin per Trade: 10 % of account
Max Losable: 1% of account



## Position Parameters

In [89]:
leverage = 300 # in multiples (x)
position_value = 30 # in currency
margin = round(position_value / leverage, 6)
margin_v_account = round(margin / starting_capital * 100, 6)
print(f"leverage: {leverage}x")
print(f"Position Value: {position_value} currency")
margin_grade = "✅" if margin_v_account <= max_margin_account_rate else "❌"
print(f"Margin: {margin} currency -> {margin_v_account}% of Account {margin_grade}")
print("")


leverage: 300x
Position Value: 30 currency
Margin: 0.1 currency -> 1.0% of Account ✅



## [Long] Position Planning

In [90]:
long_entry = 162 # in currency
long_exit = 174 # in currency
long_sl = 159 # in currency
long_position_size = round(position_value / long_entry, 6) # number of asset in the trade

long_win_pa_percentage = (long_exit - long_entry) / long_entry * 100
long_loss_pa_percentage = (long_sl - long_entry) / long_entry * 100
long_at_risk = position_value * long_loss_pa_percentage/100
long_to_win = position_value * long_win_pa_percentage/100

long_rr = round(abs(long_to_win/long_at_risk), 2)

long_at_risk_v_account = round(abs(long_at_risk/starting_capital) *100, 6)
long_to_win_v_account = round(long_to_win/starting_capital *100, 6)

print(f"Entry: {long_entry}")
print(f"Exit: {long_exit} -> +{round(long_win_pa_percentage,3)}% PA")
print(f"SL: {long_sl} -> {round(long_loss_pa_percentage,3)}% PA")
long_rr_grade_pass = True if long_rr >= rr else False
long_rr_grade = "✅" if long_rr_grade_pass else "❌"
print(f"RR: {long_rr} {long_rr_grade}")
if long_rr_grade_pass is False:
    print(f"⚠️RR goals (1:{rr}) is not met, here are suggestions")
    long_new_exit = (long_entry - long_sl) * rr + long_entry
    print(f"1/ Adjust your TP to {long_new_exit} (from {long_exit}) -> {long_new_exit - long_exit} delta")
    long_new_sl = long_entry - (long_entry - long_entry) / (rr-1)
    print(f"2/ Adjust your SL to {long_new_sl} (from {long_sl}) -> {long_new_sl - long_sl} delta")
print("")

print("---")
print(f"Capital in position: {margin} currency")
print(f"Position size: {long_position_size} assets")
long_at_risk_grade_pass = True if long_at_risk_v_account <= max_account_rate_at_risk else False
long_at_risk_grade = "✅" if long_at_risk_grade_pass else "❌"
print(f"To win: {round(long_to_win, 3)} currency -> {long_to_win_v_account}% of account")
print(f"At risk: {round(long_at_risk, 3)} currency -> {long_at_risk_v_account}% of account {long_at_risk_grade}")
if long_at_risk_grade_pass is False:
    print(f"⚠️Capital at risk goals ({max_account_rate_at_risk}%) is not met, here are suggestions")

    # new_position_value = (starting_capital * max_account_rate_at_risk / 100) * leverage
    new_position_value = (max_account_rate_at_risk / 100 * starting_capital) / abs(long_loss_pa_percentage/100)
    print(f"1/ Adjust your position size to {round(new_position_value, 2)} from {position_value} -> {new_position_value - position_value} delta")



Entry: 162
Exit: 174 -> +7.407% PA
SL: 159 -> -1.852% PA
RR: 4.0 ✅

---
Capital in position: 0.1 currency
Position size: 0.185185 assets
To win: 2.222 currency -> 22.222222% of account
At risk: -0.556 currency -> 5.555556% of account ❌
⚠️Capital at risk goals (1%) is not met, here are suggestions
1/ Adjust your position size to 5.4 from 30 -> -24.6 delta


## [Short] Position Planning

In [91]:
short_entry = 100 # in currency
short_exit = 70 # in currency
short_sl = 110 # in currency

Coming soon...